In [ ]:
from contextlib import contextmanager
import os
import sys
from joblib import Parallel, delayed
import itertools
from pdpbox.pdp_calc_utils import _calc_ice_lines_inter
from pdpbox.pdp import pdp_isolate, PDPInteract
from pdpbox.utils import (_check_model, _check_dataset, _check_percentile_range, _check_feature,
                    _check_grid_type, _check_memory_limit, _make_list,
                    _calc_memory_usage, _get_grids, _get_grid_combos, _check_classes)
import math
import time
import matplotlib.pyplot as plt
import seaborn as sns
class ML_algorithm_committee:
    def __init__(self, algorithms=None, **train_options):
        """
        Initializes the ML_algorithm_committee with the provided algorithms.
        If no algorithms are provided, use a default list of 14 algorithms.
        """
        ####attributes get from .tran_models method###
        self.models = None
        self.models_scores = None
        self.hyperparameters_optimization_results = None
        self.scaler_X = None
        self.scaler_y = None
        ###current data, once models trained, able for users to check what current model committee used for training###
        ###also, allow .importance_analysis to directly call the dataset without need to indicate X, y etc again###
        self.current_data = None
        self.problem_type = None
        self.num_classes = None
        ###attributes get from .importance_analysis###
        ###analysis type: SHAP/PDP/permutation or weighted among them, default to be SHAP###
        ###weighted/original;weighted method: r/r2 (r2 default) or accuraccy/f1-score/precision/recall/AUC for classification (AUC default)###
        self.feature_importances = None
        self.performance_ranking = None 
        self.best_models = None
        self.feature_ranking = None
        self.top_features = None
        #######################
        self.second_order_matrix=None
        ###visulization part###
        #######################
        ###scatter plot for regression prediction###
        ###AUC etc for classification prediction###
        ###output importance/ranking heatmap plot (SHAP/PDP/permutation or weighted among them, default to be SHAP) (heatmap/vionlin plot) (weighted/original) (weighted method: r/r2 etc.) feature importance and ML algorithm###
        ###default algorithms list###
        self.default_algorithm_list = ['xgboost', 'catboost', 'lightgbm', 'svm', 'knn', 'gradient_boost',
                                       'random_forest', 'decision_tree', 'extra_trees', 'adaboost',
                                       'hist_gradient_boosting', 'deepforest', 'ann', 'cnn']

    def pdp_multi_interact(self, model, dataset, model_features, features, 
                    num_grid_points=None, grid_types=None, percentile_ranges=None, grid_ranges=None, cust_grid_points=None, 
                    cust_grid_combos=None, use_custom_grid_combos=False,
                    memory_limit=0.9, n_jobs=-1, predict_kwds=None, data_transformer=None):

        if n_jobs==-1:
            n_jobs=1

        def _expand_default(x, default, length):
            if x is None:
                return [default] * length
            return x

        def _get_grid_combos(feature_grids, feature_types):
            grids = [np.array(list(feature_grid),dtype=np.float16) for feature_grid in feature_grids]
            for i in range(len(feature_types)):
                if feature_types[i] == 'onehot':
                    grids[i] = np.eye(len(grids[i])).astype(int).tolist()
            return np.stack(np.meshgrid(*grids,copy=bool), -1).reshape(-1, len(grids))

        if predict_kwds is None:
            predict_kwds = dict()

        nr_feats = len(features)

        # check function inputs
        n_classes, predict = _check_model(model=model)
        _check_dataset(df=dataset)
        _dataset = dataset.copy()

        # prepare the grid
        pdp_isolate_outs = []
        if use_custom_grid_combos:
            grid_combos = cust_grid_combos
            feature_grids = []
            feature_types = []
        else:
            num_grid_points = _expand_default(x=num_grid_points, default=5, length=nr_feats)
            grid_types = _expand_default(x=grid_types, default='percentile', length=nr_feats)
            for i in range(nr_feats):
                _check_grid_type(grid_type=grid_types[i])

            percentile_ranges = _expand_default(x=percentile_ranges, default=None, length=nr_feats)
            for i in range(nr_feats):
                _check_percentile_range(percentile_range=percentile_ranges[i])

            grid_ranges = _expand_default(x=grid_ranges, default=None, length=nr_feats)
            cust_grid_points = _expand_default(x=cust_grid_points, default=None, length=nr_feats)

            _check_memory_limit(memory_limit=memory_limit)

            pdp_isolate_outs = []
            for idx in range(nr_feats):
                pdp_isolate_out = pdp_isolate(
                    model=model, dataset=_dataset, model_features=model_features, feature=features[idx],
                    num_grid_points=num_grid_points[idx], grid_type=grid_types[idx], percentile_range=percentile_ranges[idx],
                    grid_range=grid_ranges[idx], cust_grid_points=cust_grid_points[idx], memory_limit=memory_limit,
                    n_jobs=n_jobs, predict_kwds=predict_kwds, data_transformer=data_transformer)
                pdp_isolate_outs.append(pdp_isolate_out)

            if n_classes > 2:
                feature_grids = [pdp_isolate_outs[i][0].feature_grids for i in range(nr_feats)]
                feature_types = [pdp_isolate_outs[i][0].feature_type  for i in range(nr_feats)]
            else:
                feature_grids = [pdp_isolate_outs[i].feature_grids for i in range(nr_feats)]
                feature_types = [pdp_isolate_outs[i].feature_type  for i in range(nr_feats)]

            grid_combos = _get_grid_combos(feature_grids, feature_types)

        feature_list = []
        for i in range(nr_feats):
            feature_list.extend(_make_list(features[i]))

        # Parallel calculate ICE lines
        true_n_jobs = _calc_memory_usage(
            df=_dataset, total_units=len(grid_combos), n_jobs=n_jobs, memory_limit=memory_limit)

        grid_results = Parallel(n_jobs=true_n_jobs)(delayed(_calc_ice_lines_inter)(
            grid_combo, data=_dataset, model=model, model_features=model_features, n_classes=n_classes,
            feature_list=feature_list, predict_kwds=predict_kwds, data_transformer=data_transformer)
                                                    for grid_combo in grid_combos)

        ice_lines = pd.concat(grid_results, axis=0).reset_index(drop=True)
        pdp = ice_lines.groupby(feature_list, as_index=False).mean()

        # combine the final results
        pdp_interact_params = {'n_classes': n_classes, 
                            'features': features, 
                            'feature_types': feature_types,
                            'feature_grids': feature_grids}
        if n_classes > 2:
            pdp_interact_out = []
            for n_class in range(n_classes):
                _pdp = pdp[feature_list + ['class_%d_preds' % n_class]].rename(
                    columns={'class_%d_preds' % n_class: 'preds'})
                pdp_interact_out.append(
                    PDPInteract(which_class=n_class,
                                pdp_isolate_outs=[pdp_isolate_outs[i][n_class] for i in range(nr_feats)],
                                pdp=_pdp, **pdp_interact_params))
        else:
            pdp_interact_out = PDPInteract(
                which_class=None, pdp_isolate_outs=pdp_isolate_outs, pdp=pdp, **pdp_interact_params)

        return pdp_interact_out

    def center(self, arr): 
        return arr - np.mean(arr)

    def compute_f_vals(self,mdl, X, features, selectedfeatures, num_grid_points=5, use_data_grid=False):
        f_vals = {}
        data_grid = None
        if use_data_grid:
            data_grid = X[selectedfeatures].values
        
        # check function inputs
        n_classes, predict = _check_model(model=mdl)

        # Calculate partial dependencies for full feature set
        p_full = self.pdp_multi_interact(mdl, X, features, selectedfeatures, 
                                    num_grid_points=[num_grid_points] * len(selectedfeatures),
                                    cust_grid_combos=data_grid,
                                    use_custom_grid_combos=use_data_grid)
        if n_classes >2:
            p_full_pdp=p_full[0].pdp
            for i in range(1,len(p_full)):
                p_full_pdp+=p_full[i].pdp
            p_full_pdp=p_full_pdp/n_classes
            f_vals[tuple(selectedfeatures)] = self.center(p_full_pdp.preds.values)
            grid = p_full_pdp.drop('preds', axis=1)
        else:
            f_vals[tuple(selectedfeatures)] = self.center(p_full.pdp.preds.values)
            grid = p_full.pdp.drop('preds', axis=1)
        # Calculate partial dependencies for [1..SFL-1]
        for n in range(1, len(selectedfeatures)):
            for subsetfeatures in itertools.combinations(selectedfeatures, n):
                if use_data_grid:
                    data_grid = X[list(subsetfeatures)].values
                p_partial = self.pdp_multi_interact(mdl, X, features, subsetfeatures, 
                                            num_grid_points=[num_grid_points] * len(selectedfeatures),
                                            cust_grid_combos=data_grid,
                                            use_custom_grid_combos=use_data_grid)
                
                if n_classes >2:
                    p_partial_pdp=p_partial[0].pdp
                    for i in range(1,len(p_partial)):
                        p_partial_pdp+=p_partial[i].pdp
                    p_partial_pdp=p_partial_pdp/n_classes
                    p_joined = pd.merge(grid, p_partial_pdp, how='left')
                else:
                    p_joined = pd.merge(grid, p_partial.pdp, how='left')
                f_vals[tuple(subsetfeatures)] = self.center(p_joined.preds.values)
        return f_vals

    def compute_h_val(self,f_vals, selectedfeatures):
        denom_els = f_vals[tuple(selectedfeatures)].copy()
        numer_els = f_vals[tuple(selectedfeatures)].copy()
        sign = -1.0
        for n in range(len(selectedfeatures)-1, 0, -1):
            for subfeatures in itertools.combinations(selectedfeatures, n):
                print(tuple(subfeatures))
                numer_els += sign * f_vals[tuple(subfeatures)]
            sign *= -1.0
        numer = np.sum(numer_els**2)
        denom = np.sum(denom_els**2)
        return math.sqrt(numer/denom) if numer < denom else np.nan

    def compute_h_val_any(self,f_vals, allfeatures, selectedfeature):
        otherfeatures = list(allfeatures)
        otherfeatures.remove(selectedfeature)
        denom_els = f_vals[tuple(allfeatures)].copy()
        numer_els = denom_els.copy()
        numer_els -= f_vals[(selectedfeature,)]
        numer_els -= f_vals[tuple(otherfeatures)]
        numer = np.sum(numer_els**2)
        denom = np.sum(denom_els**2)
        return math.sqrt(numer/denom) if numer < denom else np.nan

    def compute_interactions(self,model,X_train,feature_all,feature_select_list):  
        result_dict={}
        total_time = 0  # Total time taken by the function
        for i in range(len(feature_select_list)):
            for j in range(len(feature_select_list)):
                if i<j :
                    print(i,j)
                    try:
                        current_features=[feature_select_list[i],feature_select_list[j]]
                        start_time = time.time()  # Start time of the computation
                        f_vals=self.compute_f_vals(model, X_train, feature_all,current_features)
                        h_val=self.compute_h_val(f_vals,current_features)
                        elapsed_time = time.time() - start_time  # Time taken for the computation
                        if np.isnan(h_val):
                            print('current h_val is nan, convert to 0')
                            h_val = 0
                        result_dict[tuple(current_features)]=h_val
                        total_time += elapsed_time  # Add the elapsed time to the total time
                        print("Elapsed time taken:", elapsed_time)
                    except Exception as e:
                        print(e)
                        result_dict[tuple(current_features)]=0
                    print(result_dict[tuple(current_features)])
        print("Total time taken:", total_time)  # Print the total time taken
        return result_dict

    def h_index_weighted_matrix_compute(self,project_name):
        """
        Compute the H-index weighted matrix for the given models and data.

        Parameters:
        - project_name (str): The name of the project, used for creating the directory where the images are saved.

        This function computes the H-index weighted matrix for each model in the model committee. It first concatenates 
        the training and test datasets, and then computes the weighted matrix for each model based on its interactions and 
        scores. Finally, it computes the first and second order H matrices and saves them as heatmaps.

        Returns:
        - matrix_list (list): List of weighted matrices for each model.
        - second_order_matrix (DataFrame): The second order H matrix.
        - first_order_matrix (DataFrame): The first order H matrix.
        - image_1st (str): Path to the saved image of the first order H matrix.
        - image_2nd (str): Path to the saved image of the second order H matrix.
        """
        
        # Define the directory to save the images
        dir_path = os.path.join('.', project_name, 'h_index_weighted_matrix_compute')
        os.makedirs(dir_path, exist_ok=True)
        
        # Concatenate the training and test datasets
        X_for_computing= pd.concat([self.current_data[0], self.current_data[1]])

        def construct_matrix_weighted(target_dict,target_score):
            # Construct a weighted matrix from the given dictionary and score
            df=pd.DataFrame(columns=X_for_computing.columns,index=X_for_computing.columns)
            for each in target_dict:
                df.loc[each[0],each[1]]=target_dict[each]*target_score
                df.loc[each[1],each[0]]=target_dict[each]*target_score
            return df

        # Initialize the total matrix list and total metric score
        total_h_matrix_list=[]
        total_metric_score=0

        # For each model, compute the interactions and the weighted matrix, and add to the total matrix list and total metric score
        for each_key in self.models.keys():
            CURRENT_MODEL=self.models[each_key]
            CURRENT_MODEL_DICT=self.compute_interactions(CURRENT_MODEL,X_for_computing,X_for_computing.columns,list(X_for_computing.columns))
            if self.problem_type=='regression':
                CURRENT_SCORE=self.models_scores[each_key][1][1]
                CURRENT_DF=construct_matrix_weighted(CURRENT_MODEL_DICT,CURRENT_SCORE)
            else:
                CURRENT_SCORE=self.models_scores[each_key][1][4]
                CURRENT_DF=construct_matrix_weighted(CURRENT_MODEL_DICT,CURRENT_SCORE)
            total_h_matrix_list.append(CURRENT_DF)
            total_metric_score+=CURRENT_SCORE
        
        # Compute the weighted matrix
        Weighted_Matrix = sum([df.values for df in total_h_matrix_list]) / total_metric_score
        Weighted_Matrix = pd.DataFrame(Weighted_Matrix, index=total_h_matrix_list[0].index, columns=total_h_matrix_list[0].columns)
        Weighted_Matrix = Weighted_Matrix.fillna(0)
        Weighted_Matrix = Weighted_Matrix / Weighted_Matrix.max().max()
        divided_matrices = [df / total_metric_score for df in total_h_matrix_list]
        
        def Second_Order_H_Matrix_Plot(H_DF_WEIGHTED):
            # Create and save the heatmap for the second order H matrix
            # Set the figure size
            heatmap_figsize = (max(12, H_DF_WEIGHTED.shape[0] * 0.6), max(12, H_DF_WEIGHTED.shape[0] * 0.6))
            plt.figure(figsize=heatmap_figsize)
            # Create the heatmap
            sns.heatmap(H_DF_WEIGHTED, annot=False, cmap="hot_r", cbar_kws={'label': 'Second Order H Statistics'})
            # Set the title and labels
            plt.title('Weighted Second Order H statistics')
            plt.xticks(rotation=45, ha="right")
            plt.xlabel('Features')
            plt.ylabel('Features')
            plt.tight_layout()
            # Save the figure
            image_file = os.path.join(dir_path, "Second_Order_H_Matrix_Plot.png")
            plt.savefig(image_file)
            plt.show()
            return image_file

        def First_Order_H_Matrix_Plot(Model_Names_List, Matrix_list):
            # Create and save the heatmap for the first order H matrix
            # Compute the summary matrix
            summary_matrix = pd.DataFrame(index=Model_Names_List)
            for name, matrix in zip(Model_Names_List, Matrix_list):
                row_values = matrix.sum(axis=1).T  # Sum over rows
                for model_name, value in row_values.iteritems():
                    summary_matrix.loc[name, model_name] = value
            # Normalize the summary matrix
            summary_matrix = summary_matrix.fillna(0)
            summary_matrix = summary_matrix / summary_matrix.max().max()
            # Set the figure size
            heatmap_figsize = (max(12, summary_matrix.shape[0] * 0.6), max(12, summary_matrix.shape[0] * 0.6))
            plt.figure(figsize=heatmap_figsize)
            # Create the heatmap
            sns.heatmap(summary_matrix, annot=False, cmap="hot_r", cbar_kws={'label': 'First Order H Statistics'})
            # Set the title and labels
            plt.title('First Order H Statistics')
            plt.xticks(rotation=45, ha="right")
            plt.xlabel('Models')
            plt.ylabel('Models')
            plt.tight_layout()
            # Save the figure
            image_file = os.path.join(dir_path, "First_Order_H_Matrix_Plot.png")
            plt.savefig(image_file)
            plt.show()
            return summary_matrix,image_file
        
        # Compute the first and second order H matrices and save as heatmaps
        first_order_matrix,image_1st=First_Order_H_Matrix_Plot(list(self.models.keys()), divided_matrices)
        second_order_matrix=Weighted_Matrix
        image_2nd=Second_Order_H_Matrix_Plot(Weighted_Matrix)
        matrix_list=divided_matrices
        self.second_order_matrix=second_order_matrix
        plt.close()
        return matrix_list,second_order_matrix,first_order_matrix,image_1st,image_2nd


In [ ]:
import os
import shap
import pandas as pd
import keras
from sklearn.preprocessing import *
import numpy as np
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
ANN_loaded_l=keras.models.load_model("./All_HER/ANN_model_l.h5")
ANN_loaded_m=keras.models.load_model("./All_HER/ANN_model_m.h5")
ANN_loaded_h=keras.models.load_model("./All_HER/ANN_model_h.h5")

In [ ]:
def single_model_compute(ANN_loaded,project_name)
    CMT=ML_algorithm_committee()
    CMT.models={'ann':ANN_loaded}
    CMT.problem_type='regression'
    seed=1234
    Minmaxsc  = MinMaxScaler(feature_range=(0, 1))
    Minmaxsc2  = MinMaxScaler(feature_range=(0, 1))
    Stdsc  = StandardScaler()
    Stdsc2  = StandardScaler()
    MAsc  = MaxAbsScaler()
    MAsc2  = MaxAbsScaler()
    Rsc  = RobustScaler()
    Rsc2  = RobustScaler()
    database=pd.read_csv('processed_database.csv')
    data_output_full=database.iloc[:,1]
    data_input_full=database.iloc[:,2:]
    data_input_full_ANN=Stdsc.fit_transform(data_input_full)
    data_output_full_ANN=Stdsc2.fit_transform(np.array(data_output_full).reshape(-1,1))
    X_train_ANN,X_test_ANN,y_train_ANN,y_test_ANN=train_test_split(data_input_full_ANN,data_output_full_ANN,test_size=0.05,random_state=seed)
    # Getting column names
    input_column_names = data_input_full.columns
    output_column_name = data_output_full.name # Assuming data_output_full is a Series

    # Converting NumPy arrays back to DataFrame
    X_train_ANN_df = pd.DataFrame(X_train_ANN, columns=input_column_names)
    X_test_ANN_df = pd.DataFrame(X_test_ANN, columns=input_column_names)

    # If your y_train_ANN and y_test_ANN are 1-D arrays, you can convert them into a Series:
    y_train_ANN_series = pd.Series(y_train_ANN.flatten(), name=output_column_name)
    y_test_ANN_series = pd.Series(y_test_ANN.flatten(), name=output_column_name)
    CMT.current_data=(X_train_ANN_df,X_test_ANN_df,y_train_ANN_series,y_test_ANN_series)
    CMT.models_scores={'ann': ([None,1,None,None],[None,1,None,None])}
    ###H interaction compute###
    matrix_list,second_order_matrix,first_order_matrix,h_mat_image_1st,h_mat_image_2nd=CMT.h_index_weighted_matrix_compute(project_name=project_name)
    ###create H interaction directory if it doesn't exist###
    save_dir_H = f"./{project_name}/H_Compute_Save"
    os.makedirs(save_dir_H, exist_ok=True)
    ###Save the data
    np.save(os.path.join(save_dir_H, "matrix_list.npy"), matrix_list)
    np.save(os.path.join(save_dir_H, "second_order_matrix.npy"), second_order_matrix)
    np.save(os.path.join(save_dir_H, "first_order_matrix.npy"), first_order_matrix)
    return matrix_list,second_order_matrix,first_order_matrix,h_mat_image_1st,h_mat_image_2nd

In [ ]:
matrix_list_l,second_order_matrix_l,first_order_matrix_l,h_mat_image_1st_l,h_mat_image_2nd_l=single_model_compute(ANN_loaded_l,'All_HER_l')
matrix_list_m,second_order_matrix_m,first_order_matrix_m,h_mat_image_1st_m,h_mat_image_2nd_m=single_model_compute(ANN_loaded_m,'All_HER_m')
matrix_list_h,second_order_matrix_h,first_order_matrix_h,h_mat_image_1st_h,h_mat_image_2nd_h=single_model_compute(ANN_loaded_h,'All_HER_h')

In [ ]:
# # Full features
# fig, ax = plt.subplots(figsize=(5,30))
# features = first_order_matrix.loc['ann'].sort_values(ascending=False)
# colors = sns.color_palette("Blues", len(features))[::-1]  # Reverse the color palette
# sns.barplot(y=features.index, x=features.values, palette=colors, ax=ax)
# plt.title('Full features')
# plt.xlabel('Feature Importance')
# plt.ylabel('Features')
# plt.show()